In [15]:
#coding 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

# =====================================================
# 1. LOGGING SETUP
# =====================================================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("scraper.log"),
        logging.StreamHandler()
    ]
)

# =====================================================
# 2. SCRAPE WEBSITE
# =====================================================
URL = "https://pythonjobs.github.io/"

try:
    logging.info("Sending request to website...")
    response = requests.get(URL, timeout=10)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    logging.error(f"Request failed: {e}")
    raise SystemExit

soup = BeautifulSoup(response.text, "html.parser")
jobs = soup.find_all("div", class_="job")

logging.info(f"Found {len(jobs)} job listings.")

titles, companies, locations, skills, experiences = [], [], [], [], []

for idx, job in enumerate(jobs, start=1):
    try:
        title = job.find("h1")
        title = title.text.strip() if title else "Not Provided"

        company = job.find("span", class_="info")
        company = company.text.strip() if company else "Not Provided"

        location = job.find("span", class_="location")
        location = location.text.strip() if location else "Remote"

        # remove "read more" text in skill tags
        skill_tags = job.find_all("a", href=True)
        skill_list = [s.text.strip() for s in skill_tags if "read" not in s.text.lower()]
        skill_str = ", ".join(skill_list) if skill_list else "Not Mentioned"

        # this site does NOT provide experience, so we fill default
        experience = "Not Provided"

        titles.append(title)
        companies.append(company)
        locations.append(location)
        skills.append(skill_str)
        experiences.append(experience)

        logging.info(f"Scraped job {idx}: {title}")

    except Exception as e:
        logging.warning(f"Error scraping job {idx}: {e}")

# Create dataframe
df_scraped = pd.DataFrame({
    "Job Title": titles,
    "Company": companies,
    "Location": locations,
    "Skills Required": skills,
    "Experience Required": experiences
})

logging.info("Scraped DataFrame created successfully.\n")
print("WEBSITE DATA:\n", df_scraped)


# =====================================================
# 3. LOAD KAGGLE DATASET
# =====================================================
try:
    kaggle_path = r"D:\archive (2)\jobs_dataset.csv"
    kaggle_df = pd.read_csv(kaggle_path)
    logging.info("Kaggle dataset loaded successfully!\n")
except FileNotFoundError:
    logging.error(" Kaggle dataset not found at the given path!")
    raise SystemExit

logging.info(f"Kaggle Columns Found: {list(kaggle_df.columns)}")


# =====================================================
# 4. RENAME KAGGLE COLUMNS
# =====================================================
rename_map = {
    "title": "Job Title",
    "job_title": "Job Title",
    "company_name": "Company",
    "company": "Company",
    "experience": "Experience Required",
    "experience_level": "Experience Required",
    "location": "Location"
}

for col in rename_map:
    if col in kaggle_df.columns:
        kaggle_df.rename(columns={col: rename_map[col]}, inplace=True)

logging.info(f"Renamed Kaggle Columns: {list(kaggle_df.columns)}")


# =====================================================
# 5. SAFE MERGE / CONCAT
# =====================================================
common_columns = list(set(df_scraped.columns) & set(kaggle_df.columns))

if len(common_columns) >= 1:
    merged_df = pd.merge(df_scraped, kaggle_df, on=common_columns, how="outer")
    logging.info(f"Data merged on columns: {common_columns}")
else:
    merged_df = pd.concat([df_scraped, kaggle_df], ignore_index=True)
    logging.info("No common columns found. Data concatenated instead.")

# Save outputs
df_scraped.to_csv("Scraped_Jobs.csv", index=False, encoding="utf-8")
kaggle_df.to_csv("Kaggle_Cleaned.csv", index=False, encoding="utf-8")
merged_df.to_csv("Merged_Job_Data.csv", index=False, encoding="utf-8")

logging.info("CSV files saved successfully!\n")


# =====================================================
# 6. CLEAN NAN FOR GOOGLE SHEETS
# =====================================================
merged_df.replace([float("inf"), float("-inf")], "Not Provided", inplace=True)
merged_df = merged_df.fillna("Not Provided")


# =====================================================
# 7. GOOGLE SHEETS UPLOAD
# =====================================================
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

creds_path = r"D:\jobportal1-481911-767146ed231a.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)

client = gspread.authorize(creds)
sheet = client.open("Jobportal1").sheet1  # sheet must already exist

# show sheets
pprint(client.list_spreadsheet_files())

# Clear previous sheet data
sheet.clear()

# Upload new data
sheet.update([merged_df.columns.values.tolist()] + merged_df.values.tolist())

logging.info("Merged Data uploaded to Google Sheet successfully!\n")

print("\nMERGED DATA UPLOADED TO GOOGLE SHEET:\n")
print(merged_df)

2025-12-21 02:46:32,660 - INFO - Sending request to website...
2025-12-21 02:46:34,557 - INFO - Found 5 job listings.
2025-12-21 02:46:34,560 - INFO - Scraped job 1: Strats Python Developer
2025-12-21 02:46:34,565 - INFO - Scraped job 2: Python Software Developer
2025-12-21 02:46:34,568 - INFO - Scraped job 3: Senior Software Engineer, Back-End (Remote)
2025-12-21 02:46:34,576 - INFO - Scraped job 4: Python Backend Engineer
2025-12-21 02:46:34,593 - INFO - Scraped job 5: Senior Backend Engineer
2025-12-21 02:46:34,610 - INFO - Scraped DataFrame created successfully.

2025-12-21 02:46:34,811 - INFO - Kaggle dataset loaded successfully!



WEBSITE DATA:
                                      Job Title                  Company  \
0                      Strats Python Developer               London, UK   
1                    Python Software Developer          Remote, UK-only   
2  Senior Software Engineer, Back-End (Remote)  Galway, Ireland, Remote   
3                      Python Backend Engineer                   Remote   
4                      Senior Backend Engineer                   Remote   

  Location                              Skills Required Experience Required  
0   Remote                      Strats Python Developer        Not Provided  
1   Remote                    Python Software Developer        Not Provided  
2   Remote  Senior Software Engineer, Back-End (Remote)        Not Provided  
3   Remote                      Python Backend Engineer        Not Provided  
4   Remote                      Senior Backend Engineer        Not Provided  


2025-12-21 02:46:34,861 - INFO - Kaggle Columns Found: ['company', 'rating', 'location', 'positionName', 'description', 'salary', 'url', 'jobType/0', 'jobType/1', 'jobType/2', 'jobType/3', 'searchInput/country', 'searchInput/position', 'externalApplyLink']
2025-12-21 02:46:34,868 - INFO - Renamed Kaggle Columns: ['Company', 'rating', 'Location', 'positionName', 'description', 'salary', 'url', 'jobType/0', 'jobType/1', 'jobType/2', 'jobType/3', 'searchInput/country', 'searchInput/position', 'externalApplyLink']
2025-12-21 02:46:34,913 - INFO - Data merged on columns: ['Company', 'Location']
2025-12-21 02:46:35,320 - INFO - CSV files saved successfully!



[{'createdTime': '2025-12-21T09:49:21.013Z',
  'id': '168oo5k_IiJT0pY6WkZOevJKZmfwhKh6lgDj5jK641qw',
  'modifiedTime': '2025-12-21T10:39:34.242Z',
  'name': 'Jobportal1'}]


2025-12-21 02:47:01,663 - INFO - Merged Data uploaded to Google Sheet successfully!




MERGED DATA UPLOADED TO GOOGLE SHEET:

        Job Title                           Company                 Location  \
0    Not Provided                  2HB Incorporated            Chantilly, VA   
1    Not Provided                                3M                Minnesota   
2    Not Provided                   A Place For Mom             New York, NY   
3    Not Provided  A.I. Service Agents & Automation        Phoenix, AZ 85051   
4    Not Provided             ABOTTS Consulting Inc        Phoenix, AZ 85074   
..            ...                               ...                      ...   
735  Not Provided                 iHeartMedia, Inc.           New York State   
736  Not Provided                           iManage        Chicago, IL 60606   
737  Not Provided                            paypal             San Jose, CA   
738  Not Provided                      prospanceinc  Mountain View, CA 94043   
739  Not Provided                      prospanceinc  Mountain View, CA 94043   
